Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.

# Summarization using T5 model from Hugging Face

### Summarization with T5-3B model
We will use the Hugging Face Summariazion example with the T5-3B model to fine tune the model with the CNN-dailymail dataset

run_summarization.py is a lightweight example of how to download and preprocess a dataset from the 🤗 Datasets library 

#### Initial Setup
we start with a Habana PyTorch Docker image and run this notebook

#### Install Habana's DeepSpeed Fork
Habana's DeepSpeed Fork has implementations specifically for Gaudi and must be used

In [1]:
!pip install git+https://github.com/HabanaAI/DeepSpeed.git@1.10.0  

  Cloning https://github.com/HabanaAI/DeepSpeed.git (to revision 1.10.0) to /tmp/pip-req-build-lu_nb_iq
  Running command git clone --filter=blob:none --quiet https://github.com/HabanaAI/DeepSpeed.git /tmp/pip-req-build-lu_nb_iq
  Running command git checkout -b 1.10.0 --track origin/1.10.0
  Switched to a new branch '1.10.0'
  Branch '1.10.0' set up to track remote branch '1.10.0' from 'origin'.
  Resolved https://github.com/HabanaAI/DeepSpeed.git to commit 141faf783dac331bc3852590d05190dc0e883e51
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 149.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 213.7 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.7.7+hpu.synapse.v1.10.0-py3-none-any.whl size=738065 sha256=c3b2e254d4de2e9111f0fa619dc8cda0ef62b63b88c2fbee5090200ba8a7c66b
  Stored in directory: /tmp/pip-ephem-wheel-cache-qlcbq8cj/wheels/a1/1d/50/40a4b5d2dcc3710881387fe9c3712cfbc73f4b77a5

#### Install the Optimum Habana Library

In [2]:
!python -m pip install optimum[habana]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 53.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 156.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 208.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 278.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 368.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 197.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 345.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 128.9 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 217.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 134.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━

#### Clone the Hugging Face Model Repository

In [3]:
!git clone  https://github.com/huggingface/optimum-habana

Cloning into 'optimum-habana'...
remote: Enumerating objects: 4001, done.
remote: Counting objects: 100% (1154/1154), done.
remote: Compressing objects: 100% (479/479), done.
remote: Total 4001 (delta 803), reused 900 (delta 642), pack-reused 2847
Receiving objects: 100% (4001/4001), 2.05 MiB | 30.38 MiB/s, done.
Resolving deltas: 100% (2605/2605), done.


#### Go the Summarization example model and install the requirements

In [4]:
%cd optimum-habana/examples/summarization

/root/optimum-habana/examples/summarization


In [5]:
!pip install -q -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 202.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 281.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 295.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 310.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 343.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 kB 354.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.7/139.7 kB 314.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.0/412.0 kB 336.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 309.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 271.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 338.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 364.9 MB/s e

### Setup for DeepSpeed
Since we are using DeepSpeed, we have to confirm that the model has been configured properly.  We look for the following:

* deepspeed.initialize(model, ...) model, optimizer, ... = deepspeed.initialize(args=args, model=model, optimizer=optimizer, ...)
* deepspeed.init_distributed(dist_backend=“hccl”, init_method=init_method)
* Create a ds_config.json file to set the DS training parameters
  
  


#### DeepSpeed Initialization
Look in deepspeed.py and we see the model being passed to the DeepSpeed engine

In [6]:
%%sh
cd ../../optimum/habana/transformers
cat -n deepspeed.py | head -n 106 | tail -n 6
cat -n deepspeed.py | head -n 160 | tail -n 11

   101	    import deepspeed
   102	    from deepspeed.utils import logger as ds_logger
   103	
   104	    model = trainer.model
   105	    args = trainer.args
   106	
   150	    kwargs = {
   151	        "args": habana_args,
   152	        "model": model,
   153	        "model_parameters": model_parameters,
   154	        "config_params": config,
   155	        "optimizer": optimizer,
   156	        "lr_scheduler": lr_scheduler,
   157	    }
   158	
   159	    deepspeed_engine, optimizer, _, lr_scheduler = deepspeed.initialize(**kwargs)
   160	


#### DeepSpeed Distrbuted
Look in training_args.py and we see the DeepSpeed Distribution initialization

In [26]:
%%sh
cd ../../optimum/habana/transformers
cat -n training_args.py | head -n 541 | tail -n 4
cat -n training_args.py | head -n 548 | tail -n 1
cat -n training_args.py | head -n 554 | tail -n 3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
   538	            if self.deepspeed:
   539	                # deepspeed inits torch.distributed internally
   540	                from transformers.deepspeed import is_deepspeed_available
   541	
   548	
   552	
   553	                deepspeed.init_distributed(dist_backend="hccl", timeout=timedelta(seconds=self.ddp_timeout))
   554	                logger.info("DeepSpeed is enabled.")


#### Create DeepSpeed Config file with ZeRO preferences
The ds_config.json file will configure the parameters to run DeepSpeed

In this case, we will run the ZeRO3 optimier and BF16 mixed precision.

In [10]:
%%sh
tee ./ds_config.json <<EOF
{
    "steps_per_print": 64,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": "auto",
    "bf16": {
        "enabled": true
    },
    "gradient_clipping": 1.0,
    "zero_optimization": {
        "stage": 3,
        "overlap_comm": false,
        "reduce_scatter": false,
        "contiguous_gradients": false
    }
}
EOF

{
    "steps_per_print": 64,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": "auto",
    "bf16": {
        "enabled": true
    },
    "gradient_clipping": 1.0,
    "zero_optimization": {
        "stage": 3,
        "overlap_comm": false,
        "reduce_scatter": false,
        "contiguous_gradients": false
    }
}


#### Fine Tuning T5-3b with the cnn_dailymail dataset
The T5-3b model is a large language model that was origianlly trained on the C4 dataset and in this case will be fined tuned on the [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset that is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail.

For use of this model on First-Gen Gaudi, users should update the model to "T5-large"

This is run by `gaudi_spawn.py`, a simple launcher script to collect arguments and send them to `distributed_runner.py` for training on multiple HPUs, which then calls the `run_summarization.py` model.

Notice the Habana specific commands to use here:

-- use_habana  - allows training to run on Habana Gaudi  
-- use_hpu_graphs - reduces recompilation by replaying the graph  
-- gaudi_config_name Habana/t5 - mapping to Hugging Face T5 Model  

**Even though a Billion parameter T5 model can be used for Fine Tuning, this fine tuning still takes many hours to complete.  
For users that wish to execute the example Fine Tuning, they should modify the model to run "t5-small", which only takes a few hours to complete.**


In [ ]:
%cd ../summarization

In [ ]:
%%sh

mkdir ft-summarization
python ../gaudi_spawn.py \
--world_size 8 --use_deepspeed run_summarization.py \
--model_name_or_path t5-3b \
--do_train \
--do_eval \
--dataset_name cnn_dailymail \
--dataset_config '"3.0.0"' \
--source_prefix '"summarize: "' \
--output_dir ./ft-summarization \
--per_device_train_batch_size 4 \
--per_device_eval_batch_size 4 \
--overwrite_output_dir \
--predict_with_generate \
--use_habana \
--use_lazy_mode \
--use_hpu_graphs \
--gaudi_config_name Habana/t5 \
--ignore_pad_token_for_loss False \
--pad_to_max_length \
--save_strategy epoch \
--throughput_warmup_steps 3 \
--deepspeed ./ds_config.json



### After fine tuning, let's look at the results
This fine tuned model has created the new `pytorch_model.bin` and the global_step26919 folder contain the checkpoints that will be used in the infernece in the next section.


In [ ]:
%cd ./ft-summarization

In [13]:
%ls -al

total 3248
drwxrwxr-x 7 1000 1000    6144 Jun 10 10:38 ./
drwx------ 1 root root    4096 Jun 14 04:11 ../
-rw-r--r-- 1 root root     312 Jun 10 10:38 all_results.json
drwxr-xr-x 3 root root    6144 Jun 10 09:23 checkpoint-17946/
drwxr-xr-x 3 root root    6144 Jun 10 10:38 checkpoint-26919/
drwxr-xr-x 3 root root    6144 Jun 10 08:08 checkpoint-8973/
-rw-r--r-- 1 root root    1474 Jun 10 10:38 config.json
-rw-r--r-- 1 root root     506 Jun 10 10:38 gaudi_config.json
-rw-r--r-- 1 root root     142 Jun 10 10:38 generation_config.json
drwxr-xr-x 2 root root    6144 Jun 10 10:38 global_step26919/
-rw-r--r-- 1 root root      16 Jun 10 10:38 latest
drwxr-xr-x 3 root root    6144 Jun 10 06:53 runs/
-rw-r--r-- 1 root root    2201 Jun 10 10:38 special_tokens_map.json
-rw-r--r-- 1 root root  791656 Jun 10 10:38 spiece.model
-rw-r--r-- 1 root root    2324 Jun 10 10:38 tokenizer_config.json
-rw-r--r-- 1 root root 2422360 Jun 10 10:38 tokenizer.json
-rw-r--r-- 1 root root   13541 Jun 10 10:38 traine

#### Summarization using the Pipeline
Now we can run the summarization using Hugging Face Pipeline call with the fine tuned model.  In this case we will point to the mdoel that we fine tuned 

In [22]:
import torch
import habana_frameworks.torch

from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Load model to fine-tune and its tokenizer
model_to_finetune = "t5-3b"
model = AutoModelForSeq2SeqLM.from_pretrained(model_to_finetune)
tokenizer = AutoTokenizer.from_pretrained(model_to_finetune)

# Save model and tokenizer on disk
path_to_local_model = "./ft-summarization"

# Instantiate pipeline from local repo
pipe = pipeline(task="summarization", model=path_to_local_model, device="hpu", torch_dtype=torch.bfloat16)

text_to_summarize = "summarize: Introduction: The Strategic Arms Limitation Talks II (SALT II) treaty, signed on June 18, 1979, between the United States and the Soviet Union, marked a significant milestone in nuclear arms control efforts during the Cold War era. Building upon its predecessor, SALT I, the treaty aimed to curb the arms race and reduce the risk of nuclear conflict between the superpowers. Key Provisions: SALT II encompassed several crucial provisions. It placed limits on strategic offensive arms, including intercontinental ballistic missiles (ICBMs), submarine-launched ballistic missiles (SLBMs), and heavy bombers. The agreement specified the maximum number of deployed warheads and launchers each party could possess. Verification and Compliance: To ensure compliance, the treaty established comprehensive verification measures. This involved regular exchanges of data, on-site inspections, and monitoring activities by both nations. These measures sought to enhance transparency, foster trust, and prevent either side from gaining a significant advantage in terms of strategic nuclear capabilities. Ratification and Challenges: Although both the United States and the Soviet Union signed the treaty, its ratification faced considerable challenges. The political landscape changed when the Soviet Union invaded Afghanistan in 1979, leading to a deterioration of U.S.-Soviet relations. As a result, the United States never ratified the treaty formally, rendering it non-binding. However, both nations pledged to adhere to its principles, effectively implementing its provisions on a voluntary basis. Legacy and Impact: Despite the treaty's non-ratification, SALT II's legacy and impact were significant. It set the stage for subsequent arms control negotiations, providing a framework for future agreements such as the Intermediate-Range Nuclear Forces (INF) Treaty and the Strategic Arms Reduction Treaty (START). SALT II demonstrated the potential for cooperation between the superpowers and laid the groundwork for continued dialogue aimed at reducing the nuclear threat globally."
print("------------------------------------------------------------")
print("Input:", text_to_summarize)
print()

result = pipe(text_to_summarize)
print("------------------------------------------------------------")
print("Result:", result)



------------------------------------------------------------
Input: summarize: Introduction: The Strategic Arms Limitation Talks II (SALT II) treaty, signed on June 18, 1979, between the United States and the Soviet Union, marked a significant milestone in nuclear arms control efforts during the Cold War era. Building upon its predecessor, SALT I, the treaty aimed to curb the arms race and reduce the risk of nuclear conflict between the superpowers. Key Provisions: SALT II encompassed several crucial provisions. It placed limits on strategic offensive arms, including intercontinental ballistic missiles (ICBMs), submarine-launched ballistic missiles (SLBMs), and heavy bombers. The agreement specified the maximum number of deployed warheads and launchers each party could possess. Verification and Compliance: To ensure compliance, the treaty established comprehensive verification measures. This involved regular exchanges of data, on-site inspections, and monitoring activities by both nati